In [1]:
import numpy as np
import theano
import theano.tensor as T
import lasagne

import matplotlib.pyplot as plt
%matplotlib inline

import pickle
import os
from sklearn.cross_validation import train_test_split

from prepare_images import prep_image
from cnn_creator import create_cnn
from trainer import train

Using gpu device 0: GeForce GT 640M (CNMeM is disabled, cuDNN not available)


In [2]:
X = []
y = []

for cls in xrange(10):
    for fn in os.listdir('./leedsbutterfly/classes/{}'.format(cls+1)):
        _, im = prep_image('./leedsbutterfly/classes/{}/{}'.format(cls+1, fn))
        X.append(im)
        y.append(cls)
        
X = np.concatenate(X)
y = np.array(y).astype('int32')
print X.shape
print y.shape

(832, 3, 224, 224)
(832,)


In [3]:
# Split data into training, testing and validation set
rng = np.random.RandomState(0)
permutation = rng.permutation(len(X))
X, y = X[permutation], y[permutation]
train_idxs, test_idxs = train_test_split(range(len(y)),  random_state=0)
train_idxs, val_idxs = train_test_split(range(len(train_idxs)),  random_state=0)

X_train = X[train_idxs]
y_train = y[train_idxs]

X_val = X[val_idxs]
y_val = y[val_idxs]

X_test = X[test_idxs]
y_test = y[test_idxs]

print X_train.shape, y_train.shape
print X_val.shape, y_val.shape
print X_test.shape, y_test.shape
# check training data contains examples from all classes in good number
print sum(y_train==0)
print sum(y_train==1)
print sum(y_train==2)
print sum(y_train==3)
print sum(y_train==4)
print sum(y_train==5)
print sum(y_train==6)
print sum(y_train==7)
print sum(y_train==8)
print sum(y_train==9)

(468, 3, 224, 224) (468,)
(156, 3, 224, 224) (156,)
(208, 3, 224, 224) (208,)
47
47
30
52
55
55
48
32
56
46


In [4]:
net = create_cnn()

In [5]:
model, loss_history, train_acc_history, val_acc_history, predict_fn = train(net, X_train, y_train, X_val, y_val,
                                                                            batch_size=15, num_epochs=30)

training
epoch 1 / 30 : training loss: 4.841275, training accuracy: 0.383, validation loss: 4.874491, validation accuracy: 0.407
epoch 2 / 30 : training loss: 4.122441, training accuracy: 0.697, validation loss: 4.533010, validation accuracy: 0.560
epoch 3 / 30 : training loss: 3.836190, training accuracy: 0.841, validation loss: 4.460164, validation accuracy: 0.560
epoch 4 / 30 : training loss: 3.706295, training accuracy: 0.873, validation loss: 4.318395, validation accuracy: 0.600
epoch 5 / 30 : training loss: 3.494580, training accuracy: 0.951, validation loss: 4.409288, validation accuracy: 0.540
epoch 6 / 30 : training loss: 3.437674, training accuracy: 0.946, validation loss: 4.188175, validation accuracy: 0.613
epoch 7 / 30 : training loss: 3.331707, training accuracy: 0.989, validation loss: 4.199455, validation accuracy: 0.600
epoch 8 / 30 : training loss: 3.286637, training accuracy: 0.989, validation loss: 4.130254, validation accuracy: 0.647
epoch 9 / 30 : training loss: 3

In [6]:
#save the model and solver for later use
with open('saved_model2.pkl', 'wb') as output:
    pickle.dump(model, output, pickle.HIGHEST_PROTOCOL)

In [ ]:
plt.subplot(2, 1, 1)
plt.plot(np.array(loss_history).clip(max=5))
plt.xlabel('iteration')
plt.ylabel('loss')
plt.subplot(2, 1, 2)
plt.plot(train_acc_history)
plt.plot(val_acc_history)
plt.legend(['train', 'val'], loc='upper left')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.show()

In [8]:
print "best validation accuracy after training for 30 epochs: ",  max(val_acc_history)

best validation accuracy after training for 30 epochs:  0.706666666667


In [ ]:
y_test_pred = predict_fn(X_test)

In [ ]:
print np.mean(y_test==y_test_pred)